In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

C:\Users\snandanw\llm\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
### Previously we used Trainer API And Now we will use The Normal PyTorch Method for Training(finetunning)

# Prepare for training

Before actually writing our training loop, we will need to define a few objects. The first ones are the dataloaders we will use to iterate over batches. But before we can define those dataloaders, we need to apply a bit of postprocessing to our tokenized_datasets, to take care of some things that the Trainer did for us automatically. Specifically, we need to:
* 
Remove the columns corresponding to values the model does not expect (like the sentence1 and sentence2 columns
  )* .
Rename the column label to labels (because the model expects the argument to be named label
  s* ).
Set the format of the datasets so they return PyTorch tensors instead of lists.

In [4]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [5]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
print(tokenized_datasets)
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})


['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [6]:
tokenized_datasets["train"]

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

train_dataloader

In [8]:
# Checking if the DataLoader we want is properly created

for batch in train_dataloader:
    break
print(f"{batch}")
{k:v.shape for k, v in batch.items()}

{'labels': tensor([1, 1, 0, 1, 0, 1, 0, 1]), 'input_ids': tensor([[  101, 14981,  2239,  4914,  2000,  8495,  6074,  2008,  2002,  2915,
          7939,  2638, 10536,  1999,  1996,  2132,  1000,  2138,  4754,  2018,
          2699,  2000,  5607,  2032,  1010,  1000,  2429,  2000,  2019,  6545,
         10943,  2207,  9857,  1012,   102,  2429,  2000,  2019,  6545, 10943,
          2207,  9857,  1010, 14981,  2239,  4914,  2000,  8495,  6074,  2008,
          2002,  2915,  7939,  2638, 10536,  1999,  1996,  2132,  1000,  2138,
          4754,  2018,  2699,  2000,  5607,  2032,  1012,  1000,   102,     0,
             0,     0,     0,     0],
        [  101,  5405,  2001,  3517,  2000,  2022, 14374,  2013,  1996,  2902,
          9432,  2030,  5958,  1010,  2429,  2000,  9901,  2665,  1010,  1037,
          3764, 26760, 20778,  2005,  1996,  2265,  1005,  1055,  2811,  4005,
          1012,   102,  1996,  2394,  2732,  2001,  3517,  2000,  2022, 14374,
          2013,  1996,  2902,  2574

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 74]),
 'token_type_ids': torch.Size([8, 74]),
 'attention_mask': torch.Size([8, 74])}

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# To test / Check

outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6626, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [11]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\snandanw\llm\lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


### The Training Loop

In [13]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

### Evaluation


In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()